# Momentum_Strategy

In [1]:
# 设定data路径
import os

os.chdir(r'F:\Study\Study Files\05大三上\金融统计分析\实验课1\1_Rawdata')

current_path = os.getcwd()
print(current_path)

F:\Study\Study Files\05大三上\金融统计分析\实验课1\1_Rawdata


In [2]:
import pandas as pd
import numpy as np

In [3]:
data_raw = pd.read_csv('TRD_Mnth.csv')
data_raw = data_raw.drop(['Markettype'], axis=1)
data_raw['Trdmnt'] = pd.to_datetime(data_raw['Trdmnt'])

## 数据清洗

### 整体描述性统计

In [4]:
data_raw.head()

,Stkcd,Trdmnt,Msmvosd,Mretwd
0,1,1991-04-01,1157520.00,NaN
1,1,1991-05-01,1016010.00,-0.122253
2,1,1991-06-01,900735.00,-0.113459
3,1,1991-07-01,782810.00,-0.130921
4,1,1991-08-01,674833.82,-0.411588


In [5]:
des = data_raw.describe(include='all')
print(des)

               Stkcd                         Trdmnt       Msmvosd  \
count  688217.000000                         688217  6.882170e+05   
mean   328831.224049  2012-12-23 11:48:25.314457344  8.571682e+06   
min         1.000000            1990-12-01 00:00:00  1.653600e+03   
25%      2154.000000            2007-10-01 00:00:00  9.071383e+05   
50%    300470.000000            2014-08-01 00:00:00  2.348541e+06   
75%    600654.000000            2019-04-01 00:00:00  5.661909e+06   
max    900957.000000            2022-08-01 00:00:00  2.786247e+09   
std    290182.544120                            NaN  4.456034e+07   

              Mretwd  
count  683014.000000  
mean        0.013908  
min        -0.896875  
25%        -0.071429  
50%        -0.000857  
75%         0.078063  
max        22.052631  
std         0.168677  


In [6]:
print(data_raw.dtypes)

Stkcd               int64
Trdmnt     datetime64[ns]
Msmvosd           float64
Mretwd            float64
dtype: object


In [ ]:
# # 删除缺失值
# data_raw = data_raw.dropna()

# des = data_raw.describe(include='all')
# print(des)

问题：
- 每只股票第一个月的收益率为空值
- 每只股票对应的月份并非连续的，需要补充

### 检查`Stkcd`对应月份是否连续
- 结果：没有任何股票代码对应的月份是完全连续的

In [ ]:
# 创建data_result的副本
data_result_copy = data_result.copy()

# 提取Trdmnt中的月份
data_result_copy['month'] = data_result_copy['Trdmnt'].dt.month

# 定义一个函数，检查月份是否连续
def is_month_consecutive(group):
    sorted_months = group.sort_values()
    diff = np.diff(sorted_months)
    return np.all(diff == 1) or np.all(diff == -11)

# 按照Stkcd进行分组，在组内检查month是否连续
consecutive_check = data_result_copy.groupby('Stkcd')['month'].apply(is_month_consecutive)

print(consecutive_check)

# 筛选出月份连续的股票代码
consecutive_stkcd = consecutive_check[consecutive_check].index
print(consecutive_stkcd)

# 保留月份连续的股票代码的数据
data_1 = data_result_copy[data_result_copy['Stkcd'].isin(consecutive_stkcd)]

# data_1现在包含了符合条件的且月份连续的股票代码的数据
data_1.head()

### 填充数据—补充缺失月份的数据
> 逻辑：收益率取值为0，市值与上月相同

In [ ]:
# import pandas as pd
# import numpy as np

# # 确保'Trdmnt'列是datetime类型
# data_raw['Trdmnt'] = pd.to_datetime(data_raw['Trdmnt'], format='%Y-%m')

# # 对数据进行排序
# data_raw = data_raw.sort_values(by=['Stkcd', 'Trdmnt'])

# # 使用groupby和apply函数实现数据的填充
# def fill_data(df):
#     # 生成一个完整的月份范围
#     all_months = pd.date_range(df['Trdmnt'].min(), df['Trdmnt'].max(), freq='M')
    
#     # 建立一个新的DataFrame来储存完整的月份和原始数据的匹配
#     new_df = pd.DataFrame(all_months, columns=['Trdmnt'])
#     new_df['Stkcd'] = df.iloc[0]['Stkcd']
    
#     # 将新数据和旧数据合并，并填充缺失值
#     merged_df = pd.merge(new_df, df, on=['Trdmnt', 'Stkcd'], how='left')
#     merged_df['Msmvosd'].fillna(method='ffill', inplace=True) # 填充市值
#     merged_df['Mretwd'].fillna(0, inplace=True)               # 填充回报率
    
#     # 确保第一个月的回报率为0
#     merged_df.loc[merged_df['Trdmnt'] == merged_df['Trdmnt'].min(), 'Mretwd'] = 0
    
#     return merged_df

# # 应用函数并连接结果
# filled_data = pd.concat([fill_data(df) for _, df in data_raw.groupby('Stkcd')], ignore_index=True)

# # 保存处理后的数据
# filled_data.to_csv("filled_TRD_Mnth.csv", index=False)

## 策略构建

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 读取数据
data = pd.read_csv('TRD_Mnth.csv')

# 确保数据按股票代码和日期排序
data = data.sort_values(['Stkcd', 'Trdmnt'])
data['Trdmnt'] = pd.to_datetime(data['Trdmnt'])

# 设置回测期
data = data[(data['Trdmnt'] >= '1992-01-01') & (data['Trdmnt'] <= '2021-12-31')]

# 初始化一个3x3的DataFrame来保存不同策略的收益
result = pd.DataFrame(index=[1,3,6], columns=[1,3,6])

# 开始循环：不同形成期和持有期
for form_period in [1,3,6]:
    for hold_period in [1,3,6]:
        
        all_returns = []
        
        # 遍历每一个可以进行交易的日期
        for end_date in pd.date_range(start='1992-01-01', end='2021-12-31', freq='M'):
            start_date = end_date - pd.DateOffset(months=form_period)  # 形成期起始日期
            hold_end_date = end_date + pd.DateOffset(months=hold_period)  # 持有期结束日期
            
            if hold_end_date > data['Trdmnt'].max():
                continue
            
            # 计算形成期内的累计收益
            mask = (data['Trdmnt'] >= start_date) & (data['Trdmnt'] < end_date)
            form_returns = data.loc[mask].groupby('Stkcd')['Mretwd'].apply(lambda x: (1 + x).prod() - 1)

            # 检查在持有期间是否有数据
            mask = (data['Trdmnt'] > end_date) & (data['Trdmnt'] <= hold_end_date)
            available_stocks = data.loc[mask]['Stkcd'].unique()
            
            # 仅保留那些在持有期内有数据的股票
            form_returns = form_returns[form_returns.index.isin(available_stocks)]
            
            # 从剩下的股票中确定赢家和输家
            
            winners = form_returns.nlargest(10).index
            losers = form_returns.nsmallest(10).index
            
            # 计算持有期收益
            mask = (data['Trdmnt'] > end_date) & (data['Trdmnt'] <= hold_end_date)
            hold_returns = data.loc[mask].groupby(['Stkcd', 'Trdmnt'])['Mretwd'].apply(lambda x: ((1 + x).prod())**(1/hold_period) - 1).unstack().mean(axis=1)
            
            winner_returns = hold_returns.loc[winners].mean()
            loser_returns = hold_returns.loc[losers].mean()
            
            # 计算策略收益：赢家收益 - 输家收益
            strategy_return = winner_returns - loser_returns
            
            all_returns.append(strategy_return)
            
        cumulative_returns = all_returns + 1  # 每个月的累计收益率
        geometric_mean = np.prod(cumulative_returns) - 1  # 几何平均收益率
        annual_returns = (geometric_mean + 1) ** 12 - 1  # 年收益率

        result.loc[form_period, hold_period] = annual_returns


## 完整代码

In [ ]:
import os

# 获取当前工作目录
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# 使用相对路径定位到目标目录
target_dir = os.path.join(current_dir, '..', '1_Rawdata')

# 更改工作目录
os.chdir(target_dir)
print(f"Target working directory: {os.getcwd()}")

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

# 读取数据
data = pd.read_csv('TRD_Mnth.csv')
data = data.drop(['Markettype'], axis=1)

forming_month_j = [1, 3, 6]
holding_month_k = [1, 3, 6]

trade_begin = '2000-01-01'
trade_end = '2010-12-31'

# 确保数据按股票代码和日期排序
data = data.sort_values(['Stkcd', 'Trdmnt'])
data['Trdmnt'] = pd.to_datetime(data['Trdmnt'])

# 设置回测期
data_trade = data[(data['Trdmnt'] >= trade_begin) & (data['Trdmnt'] <= trade_end)]

# 将Msmvosd缺失的数据剔除
data = data[data['Msmvosd'].notnull()]

# 初始化一个DataFrame来保存不同策略的收益
result = pd.DataFrame(index=forming_month_j, columns=holding_month_k)
result_weighted = pd.DataFrame(index=forming_month_j, columns=holding_month_k)

# 获取唯一的年月，作为回测的月份点
unique_year_month = pd.PeriodIndex(data_trade['Trdmnt'].dt.to_period('M').unique())

# 开始循环：不同形成期和持有期
for form_period in forming_month_j:
    for hold_period in holding_month_k:

        all_returns = []
        all_returns_winners = []
        all_returns_losers = []
        past_winners = []
        past_losers = []

        all_returns_weighted = []
        all_returns_winners_weighted = []
        all_returns_losers_weighted = []

        # 根据hold_period生成额外的月份
        extra_months = pd.PeriodIndex([unique_year_month[-1] + i for i in range(1, hold_period)], freq='M')
        full_year_month = unique_year_month.union(extra_months)

        # 遍历每一个唯一的年月，包括额外的月份
        for current_ym in full_year_month:
            current_ym_str = str(current_ym)
            current_ym_datetime = pd.to_datetime(current_ym.to_timestamp()).to_pydatetime()
            if current_ym_datetime <= unique_year_month.max().to_timestamp():
                
                end_date = pd.to_datetime(current_ym.to_timestamp())
                start_date = end_date - pd.DateOffset(months=form_period)

                mask = (data['Trdmnt'] > start_date) & (data['Trdmnt'] <= end_date)
                form_returns = data.loc[mask].groupby('Stkcd')['Mretwd'].apply(lambda x: (1 + x).prod() - 1)

                available_stocks = data.loc[data['Trdmnt'] > end_date]['Stkcd'].unique()
                form_returns = form_returns[form_returns.index.isin(available_stocks)]

                #将股票按form_returns大小分为5组
                groups_form_returns = pd.qcut(form_returns, 5, labels=False)

                #赢家股票为form_returns最高的1组
                winners = form_returns[groups_form_returns == 4].index.tolist()

                #输家股票为form_returns最低的1组
                losers = form_returns[groups_form_returns == 0].index.tolist()

                past_winners.append(winners)
                past_losers.append(losers)

                # 如果我们的列表变得过长，就移除旧的元素
                if len(past_winners) > hold_period:
                    past_winners.pop(0)
                    past_losers.pop(0)
            else:
                # 如果不在交易期内，则继续移除旧的元素
                past_winners.pop(0)
                past_losers.pop(0)
            
            # 计算当月所有past_winners和past_losers的平均收益
            mask = (data['Trdmnt'] == str(current_ym))
            current_winner_returns = [
                data.loc[mask & data['Stkcd'].isin(winners_month)]['Mretwd'].mean() 
                for winners_month in past_winners
            ]
            current_loser_returns = [
                data.loc[mask & data['Stkcd'].isin(losers_month)]['Mretwd'].mean() 
                for losers_month in past_losers
            ]
            
            # 取算数平均
            current_winner_return = np.mean(current_winner_returns)
            current_loser_return = np.mean(current_loser_returns)

            # 取加权平均
            current_winner_returns_weighted = [
                np.average(
                    data.loc[mask & data['Stkcd'].isin(winners_month) & data['Mretwd'].notna() & data['Msmvosd'].notna()]['Mretwd'], 
                    weights=data.loc[mask & data['Stkcd'].isin(winners_month) & data['Mretwd'].notna() & data['Msmvosd'].notna()]['Msmvosd']
                )
                for winners_month in past_winners
            ]
            current_loser_returns_weighted = [
                np.average(
                    data.loc[mask & data['Stkcd'].isin(losers_month) & data['Mretwd'].notna() & data['Msmvosd'].notna()]['Mretwd'], 
                    weights=data.loc[mask & data['Stkcd'].isin(losers_month) & data['Mretwd'].notna() & data['Msmvosd'].notna()]['Msmvosd']
                )
                for losers_month in past_losers
            ]

            # 取加权平均
            current_winner_return_weighted = np.nanmean(current_winner_returns_weighted)
            current_loser_return_weighted = np.nanmean(current_loser_returns_weighted)

            # 计算策略收益：赢家收益 - 输家收益
            #strategy_return = current_winner_return - current_loser_return
            all_returns_winners.append(current_winner_return)
            all_returns_losers.append(current_loser_return)

            all_returns_winners_weighted.append(current_winner_return_weighted)
            all_returns_losers_weighted.append(current_loser_return_weighted)

            all_returns.append(current_winner_return - current_loser_return)
            all_returns_weighted.append(current_winner_return_weighted - current_loser_return_weighted)

        # def calculate_annual_returns(all_returns):
        #     all_returns_clean = [x for x in all_returns if not np.isnan(x)]
        #     cumulative_returns = [x + 1 for x in all_returns_clean]
        #     geometric_mean = np.prod(cumulative_returns)**(1/len(cumulative_returns)) - 1
        #     annual_returns = (geometric_mean + 1) ** 12 - 1
        #     return annual_returns
        
        def calculate_geometric_mean_returns(all_returns):
            all_returns_clean = [x for x in all_returns if not np.isnan(x)]
            cumulative_returns = [x + 1 for x in all_returns_clean]
            geometric_mean_returns = np.prod(cumulative_returns)**(1/len(cumulative_returns)) - 1
            return geometric_mean_returns

        # annual_returns = calculate_annual_returns(all_returns)
        # annual_returns_weighted = calculate_annual_returns(all_returns_weighted)

        geometric_mean_returns = calculate_geometric_mean_returns(all_returns)
        geometric_mean_returns_weighted = calculate_geometric_mean_returns(all_returns_weighted)

        result.loc[form_period, hold_period] = geometric_mean_returns
        result_weighted.loc[form_period, hold_period] = geometric_mean_returns_weighted

print(result)
print(result_weighted)

import matplotlib.pyplot as plt
import seaborn as sns

# 绘制热力图
sns.heatmap(result.astype(float), annot=True, fmt=".2%", cmap="vlag", center=0)
title_line1 = "Momentum Strategy Returns (Jegadeesh and Titman, 1993)"
title_line2 = "Unweighted Average"
title_line3 = trade_begin + "~" + trade_end
plt.title(f"{title_line1}\n{title_line2}\n{title_line3}")
plt.xlabel("Holding Period (months)")
plt.ylabel("Formation Period (months)")

# 保存图像到'3_Output'文件夹
output_pic_file_name = "Momentum_Strategy_heatmap_" + trade_begin + "~" + trade_end + ".png"
trading_period = trade_begin + "~" + trade_end
output_folder = f'../3_Output/{trading_period}'
os.makedirs(output_folder, exist_ok=True)
output_pic_file_name = "Momentum_Strategy_heatmap_" + trading_period + ".png"
plt.savefig(f'{output_folder}/{output_pic_file_name}')

# 显示图像
plt.show()

import matplotlib.pyplot as plt
import seaborn as sns

# 绘制热力图
sns.heatmap(result_weighted.astype(float), annot=True, fmt=".2%", cmap="vlag", center=0)
title_line1 = "Momentum Strategy Returns (Jegadeesh and Titman, 1993)"
title_line2 = "Weighted Average"
title_line3 = trade_begin + "~" + trade_end
plt.title(f"{title_line1}\n{title_line2}\n{title_line3}")
plt.xlabel("Holding Period (months)")
plt.ylabel("Formation Period (months)")

# 保存图像到'3_Output'文件夹
output_pic_file_name = "Momentum_Strategy_heatmap_weighted_" + trade_begin + "~" + trade_end + ".png"
trading_period = trade_begin + "~" + trade_end
output_folder = f'../3_Output/{trading_period}'
os.makedirs(output_folder, exist_ok=True)
output_pic_file_name = "Momentum_Strategy_heatmap_weighted_" + trading_period + ".png"
plt.savefig(f'{output_folder}/{output_pic_file_name}')

# 显示图像
plt.show()